In [ ]:
# ✅ CSV 파일 읽기
df = try_read_csv(file_path)
additional_df = try_read_csv(additional_file_path)

# ✅ 컬럼 리스트 출력
print("✅ [df] 주요 파일 컬럼 목록:")
print(df.columns.tolist())
print("\n✅ [additional_df] 추가 병합용 파일 컬럼 목록:")
print(additional_df.columns.tolist())

NameError: name 'try_read_csv' is not defined

In [ ]:
# 예시) KTT월정료(조정) 존재 여부에 따른 rename 처리
if 'KTT월정료(조정)' in df.columns:
    df = df.rename(columns={'KTT월정료(조정)': 'KTT월정료'})
elif 'KTT월정료' not in df.columns:
    raise KeyError("❌ 'KTT월정료(조정)' 또는 'KTT월정료' 컬럼이 존재하지 않습니다.")

In [ ]:
# ✅ 1. 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 모듈 불러오기
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar

# ✅ 3. 파일 경로 설정 (Google Drive 기준)
file_path = '/content/drive/MyDrive/시설/정지/C150_G0000_00013.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0422_전사 정지조건리스트_완성.xlsx'
output_cs_path = '/content/drive/MyDrive/시설/정지/2025년0422_전사 정지조건리스트_완성.csv'

# ✅ 4. 인코딩 자동 감지 및 CSV 읽기
def try_read_csv(path):
    encodings = ['utf-8-sig', 'cp949', 'euc-kr']
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except:
            continue
    raise ValueError(f"❌ 파일을 열 수 없습니다: {path}")

df = try_read_csv(file_path)
additional_df = try_read_csv(additional_file_path)

# ✅ 5. 컬럼명 확인 및 정제
columns_to_extract = [
    '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
    '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료', '계약시작일', '계약종료일',
    '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
]

df = df[[col for col in columns_to_extract if col in df.columns]]
df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

# ✅ 6. 추가 컬럼 병합 (컬럼 존재 여부 체크 포함)
if 'KTT월정료(조정)' in additional_df.columns:
    additional_df = additional_df.rename(columns={'KTT월정료(조정)': 'KTT월정료'})

additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소', 'KTT월정료']
valid_additional_columns = [col for col in additional_columns if col in additional_df.columns]

additional_df = additional_df[['계약번호'] + valid_additional_columns].drop_duplicates(subset='계약번호')
for col in valid_additional_columns:
    df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

for col in additional_columns:
    if col not in df.columns:
        df[col] = ''

# ✅ 7. 제외사유 제거
df = df[df['제외사유'].astype(str).str.strip() == '']

# ✅ 8. 날짜 정제
def convert_date(x):
    x = str(x).strip()
    if len(x) >= 8 and x.isdigit():
        return f"{x[:4]}-{x[4:6]}-{x[6:8]}"
    return None

for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
    df.loc[:, col] = df[col].apply(convert_date)

# ✅ 9. 정지희망종료일 보정
today = datetime.today()
last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
last_day_str = last_day.strftime("%Y-%m-%d")
df.loc[:, '정지희망종료일'] = df['정지희망종료일'].apply(
    lambda x: last_day_str if not x or x == '9999-12-31' else x)

# ✅ 10. 정지일수 계산
def calc_days(s, e):
    try:
        s = datetime.strptime(s, "%Y-%m-%d")
        e = datetime.strptime(e, "%Y-%m-%d")
        return max((e - s).days + 1, 0)
    except:
        return None

df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

# ✅ 11. 정지일수 구간 분류
def categorize_freeze(days):
    if pd.isna(days): return None
    elif days <= 89: return '89일이하'
    elif days <= 119: return '90~119일'
    elif days <= 149: return '120~149일'
    else: return '150일이상'

df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze)

# ✅ 12. 종료희망일 초과 여부
df['종료희망일 초과 여부'] = df['정지희망종료일'].apply(
    lambda d: '초과' if pd.to_datetime(d, errors='coerce') <= last_day else '미초과'
)

# ✅ 13. 월정료 구간
def fee_range(val):
    try:
        v = float(str(val).replace(',', ''))
        if v <= 50000: return '5만원 이하'
        elif v <= 70000: return '5만원 초과 ~ 7만원 이하'
        elif v <= 100000: return '7만원 초과 ~ 10만원 이하'
        elif v <= 200000: return '10만원 초과 ~ 20만원 이하'
        else: return '20만원 초과'
    except:
        return None

df['월정료 구간'] = df['KTT월정료'].apply(fee_range)

# ✅ 14. 컬럼 순서 정리
new_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
df = df[[col for col in new_columns if col in df.columns]]

# ✅ 15. 엑셀 저장 (노란색 강조)
df.to_excel(output_excel_path, index=False, engine='openpyxl')
wb = load_workbook(output_excel_path)
ws = wb.active
yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
for col in range(1, len(additional_columns) + 5):
    for row in range(2, ws.max_row + 1):
        ws.cell(row=row, column=col).fill = yellow_fill
wb.save(output_excel_path)

# ✅ 16. CSV 저장 (cp949: 윈도우 한글 호환)
df.to_csv(output_cs_path, index=False, encoding='cp949')

print(f"✅ 저장 완료! 최종 행 수: {df.shape[0]}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-10-ea2f365143bb>:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2025-04-01' '2024-11-30' '2024-10-24' ... '2025-03-31' '2025-03-31'
 '2025-03-31']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].apply(convert_date)
<ipython-input-10-ea2f365143bb>:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2026-03-31' '2025-11-29' '2025-10-23' ... '2026-03-30' '2030-03-30'
 '2029-03-30']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].apply(convert_date)
<ipython-input-10-ea2f365143bb>:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2024-10-31' '2025-04-01' '2025-04-01' ... '2025-04-15' '2025-04-02'
 '2025-04-01']' has dtype in

✅ 저장 완료! 최종 행 수: 5329


In [ ]:
## 쥬피터노트북용(회사)

# ✅ 1. 필수 모듈 불러오기
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import os

# ✅ 2. 파일 경로 설정 (Windows 경로)
file_path = r'D:\시설\정지\일일정지\20250422\C150_G0000_00013.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.xlsx'
output_cs_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.csv'

# ✅ 3. CSV 읽기 함수 (인코딩 자동 감지)
def try_read_csv(path):
    encodings = ['utf-8-sig', 'cp949', 'euc-kr']
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except:
            continue
    raise ValueError(f"❌ 파일을 열 수 없습니다: {path}")

# ✅ 4. 데이터 로드
df = try_read_csv(file_path)
additional_df = try_read_csv(additional_file_path)

# ✅ 5. 주요 컬럼 필터링
columns_to_extract = [
    '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
    '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', '계약시작일', '계약종료일',
    '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
]
df = df[[col for col in columns_to_extract if col in df.columns]]
df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

# ✅ 6. 병합 컬럼 준비
if 'KTT월정료(조정)' in additional_df.columns:
    additional_df = additional_df.rename(columns={'KTT월정료(조정)': 'KTT월정료'})

additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소', 'KTT월정료']
valid_additional_columns = [col for col in additional_columns if col in additional_df.columns]

# ✅ 7. 계약번호 기준 병합
additional_df = additional_df[['계약번호'] + valid_additional_columns].drop_duplicates(subset='계약번호')
for col in valid_additional_columns:
    df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

# ✅ 8. 없는 컬럼은 빈값 처리
for col in additional_columns:
    if col not in df.columns:
        df[col] = ''

# ✅ 9. 제외사유 제거
df = df[df['제외사유'].astype(str).str.strip() == '']

# ✅ 10. 날짜 포맷 함수
def convert_date(x):
    x = str(x).strip()
    if len(x) >= 8 and x.isdigit():
        return f"{x[:4]}-{x[4:6]}-{x[6:8]}"
    return None

for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
    if col in df.columns:
        df[col] = df[col].apply(convert_date)

# ✅ 11. 정지희망종료일 보정
today = datetime.today()
last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
last_day_str = last_day.strftime("%Y-%m-%d")
df['정지희망종료일'] = df['정지희망종료일'].apply(lambda x: last_day_str if not x or x == '9999-12-31' else x)

# ✅ 12. 정지일수 계산
def calc_days(s, e):
    try:
        s = datetime.strptime(s, "%Y-%m-%d")
        e = datetime.strptime(e, "%Y-%m-%d")
        return max((e - s).days + 1, 0)
    except:
        return None

df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

# ✅ 13. 정지일수 구간 분류
def categorize_freeze(days):
    if pd.isna(days): return None
    elif days <= 89: return '89일이하'
    elif days <= 119: return '90~119일'
    elif days <= 149: return '120~149일'
    else: return '150일이상'
df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze)

# ✅ 14. 종료희망일 초과 여부
df['종료희망일 초과 여부'] = df['정지희망종료일'].apply(
    lambda d: '초과' if pd.to_datetime(d, errors='coerce') <= last_day else '미초과'
)

# ✅ 15. 월정료 구간 계산
def fee_range(val):
    try:
        v = float(str(val).replace(',', ''))
        if v <= 50000: return '5만원 이하'
        elif v <= 70000: return '5만원 초과 ~ 7만원 이하'
        elif v <= 100000: return '7만원 초과 ~ 10만원 이하'
        elif v <= 200000: return '10만원 초과 ~ 20만원 이하'
        else: return '20만원 초과'
    except:
        return None
df['월정료 구간'] = df['KTT월정료'].apply(fee_range)

# ✅ 16. 컬럼 순서 정리
final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
df = df[[col for col in final_columns if col in df.columns]]

# ✅ 17. Excel 저장 및 노란색 강조
df.to_excel(output_excel_path, index=False, engine='openpyxl')

wb = load_workbook(output_excel_path)
ws = wb.active
yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
for col in range(1, len(additional_columns) + 5):
    for row in range(2, ws.max_row + 1):
        ws.cell(row=row, column=col).fill = yellow_fill
wb.save(output_excel_path)

# ✅ 18. CSV 저장
df.to_csv(output_cs_path, index=False, encoding='cp949')

print(f"✅ 저장 완료! 최종 행 수: {df.shape[0]}")

ValueError: ❌ 파일을 열 수 없습니다: D:\시설\정지\일일정지\20250422\C150_G0000_00013.csv

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet

# ✅ 인코딩 감지 함수 (euc-kr → cp949 자동 대체)
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250422\C150_G0000_00013.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.csv'

try:
    # ✅ 파일 불러오기 (인코딩 자동 감지)
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', '계약시작일', '계약종료일',
        '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 처리
    if 'KTT월정료(조정)' in df.columns:
        df = df.rename(columns={'KTT월정료(조정)': 'KTT월정료'})
    if 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 날짜 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간 분류
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간 분류
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만원 이하'
            elif amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 음영 강조 (추가컬럼 및 계산컬럼 강조)
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"✅ 최종 저장 완료! (필터링 후 행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 파일 경로 설정
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_cs_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 통일: 'KTT월정료(조정)' → 'KTT월정료' 우선 적용
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''  # 없는 경우 빈 컬럼 생성

    # ✅ 날짜 포맷 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간 분류
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만원 이하'
            elif amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글 호환용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"✅ 최종 저장 완료! (행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)

In [ ]:
# ✅ 2. 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')



import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?()\-/:;@&\'"※•·ㆍ→←↑↓▶★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 파일 경로 설정
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ 계약번호 중복 제거 및 KTT월정료(조정) 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 날짜 정제
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만원 이하'
            elif amount <= 70000:
                return '5만~7만'
            elif amount <= 100000:
                return '7만~10만'
            elif amount <= 200000:
                return '10만~20만'
            else:
                return '20만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 특수문자 정제 후 저장
    df = clean_text_columns(df)

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (utf-8-sig로 안전 저장)
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"✅ 최종 저장 완료! (행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)

In [ ]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 라이브러리 임포트
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re
import os

# ✅ 3. 인코딩 자동 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 4. 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?:;()\[\]{}@&%~\-+=/\\_|\'"<>※•·ㆍ→←↑↓▶☆★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 5. 경로 설정 (사용자 환경에 맞게 조정)
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 6. 파일 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 7. 주요 컬럼 필터링 및 서비스 조건 적용
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 8. 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 9. 추가 데이터 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]
    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 10. 제외사유 필터링
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ 11. 계약번호 기준 중복 제거 및 월정료 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 12. 날짜 정제
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 13. 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(lambda x: last_day_str if not x or x == '9999-12-31' else x)

    # ✅ 14. 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 15. 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 16. 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 17. 월정료 구간
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만 이하'
            elif amount <= 70000:
                return '5만~7만'
            elif amount <= 100000:
                return '7만~10만'
            elif amount <= 200000:
                return '10만~20만'
            else:
                return '20만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 18. 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 19. 특수문자 제거
    df = clean_text_columns(df)

    # ✅ 20. 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 21. 강조 색상 적용
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ 22. CSV 저장 (Excel에서도 열리는 UTF-8 with BOM)
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"엑셀 경로: {output_excel_path}")
    print(f"CSV 경로: {output_csv_path}")

except Exception as e:
    print("\n❌ 오류 발생:", e)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-3-e2e990e0d220>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str).apply(convert_date)



❌ 오류 발생: expected string or bytes-like object, got 'Series'


In [ ]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 라이브러리 로드
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re
import os

# ✅ 3. 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 4. 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?:;()\[\]{}@&%~\-+=/\\_|\'"<>※•·ㆍ→←↑↓▶☆★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 5. 파일 경로 설정
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 6. 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 7. 컬럼 필터 및 조건 적용
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 8. 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 9. 추가 데이터 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]
    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 10. 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ 11. 중복 제거 및 KTT월정료 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 12. 날짜 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df.loc[:, col] = df[col].apply(lambda x: convert_date(x))

    # ✅ 13. 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(lambda x: last_day_str if not x or x == '9999-12-31' else x)

    # ✅ 14. 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 15. 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 16. 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 17. 월정료 구간
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만 이하'
            elif amount <= 70000:
                return '5만~7만'
            elif amount <= 100000:
                return '7만~10만'
            elif amount <= 200000:
                return '10만~20만'
            else:
                return '20만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 18. 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 19. 특수문자 제거
    df = clean_text_columns(df)

    # ✅ 20. 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 21. 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ 22. CSV 저장 (한글 깨짐 없이 Excel 호환)
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"엑셀 저장 경로: {output_excel_path}")
    print(f"CSV 저장 경로: {output_csv_path}")

except Exception as e:
    print("\n❌ 오류 발생:", e)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-26feb4f2e1f6>:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2025-04-01' '2024-11-30' '2024-10-24' ... '2025-03-31' '2025-03-31'
 '2025-04-24']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].apply(lambda x: convert_date(x))
<ipython-input-4-26feb4f2e1f6>:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2026-03-31' '2025-11-29' '2025-10-23' ... '2030-03-30' '2029-03-30'
 '2029-04-23']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].apply(lambda x: convert_date(x))
<ipython-input-4-26feb4f2e1f6>:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2024-10-31' '2025-04-01' '2025-04-01' ... '2025-04-02' '2025-04-01'
 '202


❌ 오류 발생: expected string or bytes-like object, got 'Series'


<ipython-input-4-26feb4f2e1f6>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str).apply(
<ipython-input-4-26feb4f2e1f6>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str).apply(
<ipython-input-4-26feb4f2e1f6>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [ ]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 라이브러리 로드
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re
import os

# ✅ 3. 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 4. 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df.loc[:, col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?:;()\[\]{}@&%~\-+=/\\_|\'"<>※•·ㆍ→←↑↓▶☆★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 5. 경로 설정
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 6. 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 7. 컬럼 필터링 및 조건 적용
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 8. 본부명 정제
    df.loc[:, '관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 9. 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]
    for col in additional_columns:
        df.loc[:, col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 10. 제외사유 제거
    if '제외사유' in df.columns:
        df = df[df['제외사유'].astype(str).str.strip() == '']

    # ✅ 11. 계약번호 중복 제거 + 월정료 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 12. 날짜 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df.loc[:, col] = df[col].astype(str).apply(convert_date)

    # ✅ 13. 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(lambda x: last_day_str if not x or x == '9999-12-31' else x)

    # ✅ 14. 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 15. 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 16. 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 17. 월정료 구간
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만 이하'
            elif amount <= 70000:
                return '5만~7만'
            elif amount <= 100000:
                return '7만~10만'
            elif amount <= 200000:
                return '10만~20만'
            else:
                return '20만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 18. 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 19. 특수문자 제거
    df = clean_text_columns(df)

    # ✅ 20. 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 21. 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ 22. CSV 저장 (Excel 한글 호환용)
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"엑셀 저장 경로: {output_excel_path}")
    print(f"CSV 저장 경로: {output_csv_path}")

except Exception as e:
    print("\n❌ 오류 발생:", e)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-5-a8ae11e8ba68>:94: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2025-04-01' '2024-11-30' '2024-10-24' ... '2025-03-31' '2025-03-31'
 '2025-04-24']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].astype(str).apply(convert_date)
<ipython-input-5-a8ae11e8ba68>:94: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2026-03-31' '2025-11-29' '2025-10-23' ... '2030-03-30' '2029-03-30'
 '2029-04-23']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].astype(str).apply(convert_date)
<ipython-input-5-a8ae11e8ba68>:94: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2024-10-31' '2025-04-01' '2025-04-01' ... '2025-04-02' '2025-04-01'
 '2025-


❌ 오류 발생: expected string or bytes-like object, got 'Series'


In [ ]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 라이브러리 로드
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re

# ✅ 3. 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 4. 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df.loc[:, col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?:;()\[\]{}@&%~\-+=/\\_|\'"<>※•·ㆍ→←↑↓▶☆★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 5. 날짜 정제 함수
def convert_date(value):
    try:
        value = str(value).strip()
        if not value or pd.isna(value):
            return None
        if len(value) == 8 and value.isdigit():
            return f"{value[:4]}-{value[4:6]}-{value[6:8]}"
        return value if '-' in value else None
    except:
        return None

# ✅ 6. 경로 설정
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 7. 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 8. 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]].copy()
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 9. 본부명 정제
    df.loc[:, '관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 10. 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df.loc[:, col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 11. 제외사유 제거
    if '제외사유' in df.columns:
        df = df[df['제외사유'].astype(str).str.strip() == '']

    # ✅ 12. 계약번호 기준 중복 제거 + 월정료 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 13. 날짜 정제 적용
    date_cols = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
    for col in date_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 14. 정지희망종료일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime('%Y-%m-%d')
    df.loc[:, '정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 15. 종료희망일 초과 여부
    df.loc[:, '종료희망일 초과 여부'] = df['정지희망종료일'].apply(
        lambda x: '초과' if pd.to_datetime(x, errors='coerce') <= last_day else '미초과'
    )

    # ✅ 16. 정지일수 계산
    def calc_days(start, end):
        try:
            s = pd.to_datetime(start, errors='coerce')
            e = pd.to_datetime(end, errors='coerce')
            return max((e - s).days + 1, 0)
        except:
            return None
    df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 17. 정지일수 구간
    df['정지일수 구간'] = pd.cut(df['정지일수'], bins=[-1, 89, 119, 149, float('inf')], labels=['89일이하', '90~119일', '120~149일', '150일이상'])

    # ✅ 18. 월정료 구간
    def fee_group(val):
        try:
            v = float(str(val).replace(',', ''))
            if v <= 50000: return '5만 이하'
            elif v <= 70000: return '5만~7만'
            elif v <= 100000: return '7만~10만'
            elif v <= 200000: return '10만~20만'
            else: return '20만 초과'
        except:
            return None
    df['월정료 구간'] = df['KTT월정료'].apply(fee_group)

    # ✅ 19. 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 20. 특수문자 제거
    df = clean_text_columns(df)

    # ✅ 21. 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 22. 노란색 강조 처리
    from openpyxl import load_workbook
    wb = load_workbook(output_excel_path)
    ws = wb.active
    fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = fill
    wb.save(output_excel_path)

    # ✅ 23. CSV 저장
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"엑셀 저장 경로: {output_excel_path}")
    print(f"CSV 저장 경로: {output_csv_path}")

except Exception as e:
    print("\n❌ 오류 발생:", e)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-6-a73e998378af>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str).apply(convert_date)
<ipython-input-6-a73e998378af>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str).apply(convert_date)
<ipython-input-6-a73e998378af>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa


❌ 오류 발생: expected string or bytes-like object, got 'Series'


In [ ]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 필수 라이브러리
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re

# ✅ 3. 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 4. 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df.loc[:, col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?:;()\[\]{}@&%~\-+=/\\_|\'"<>※•·ㆍ→←↑↓▶☆★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 5. 날짜 변환 함수
def convert_date(value):
    try:
        value = str(value).strip()
        if not value or pd.isna(value):
            return None
        if len(value) == 8 and value.isdigit():
            return f"{value[:4]}-{value[4:6]}-{value[6:8]}"
        return value if '-' in value else None
    except:
        return None

# ✅ 6. 파일 경로 설정
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 7. 파일 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 8. 필터링 및 정제
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]].copy()
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 9. 본부명 통합
    df.loc[:, '관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 10. 추가 정보 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    for col in additional_columns:
        df.loc[:, col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 11. 제외사유 제거
    df = df[df['제외사유'].astype(str).str.strip() == '']

    # ✅ 12. 계약번호 기준 중복 제거 및 월정료 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 13. 날짜 정제 (FutureWarning 제거)
    date_cols = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
    for col in date_cols:
        if col in df.columns:
            df[col] = df[col].astype("object").apply(convert_date)

    # ✅ 14. 정지희망종료일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime('%Y-%m-%d')
    df.loc[:, '정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 15. 종료희망일 초과 여부
    df['종료희망일 초과 여부'] = df['정지희망종료일'].apply(
        lambda x: '초과' if pd.to_datetime(x, errors='coerce') <= last_day else '미초과'
    )

    # ✅ 16. 정지일수 계산
    def calc_days(start, end):
        try:
            s = pd.to_datetime(start, errors='coerce')
            e = pd.to_datetime(end, errors='coerce')
            return max((e - s).days + 1, 0)
        except:
            return None
    df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 17. 정지일수 구간
    df['정지일수 구간'] = pd.cut(df['정지일수'], bins=[-1, 89, 119, 149, float('inf')],
                                labels=['89일이하', '90~119일', '120~149일', '150일이상'])

    # ✅ 18. 월정료 구간
    def fee_group(val):
        try:
            v = float(str(val).replace(',', ''))
            if v <= 50000: return '5만 이하'
            elif v <= 70000: return '5만~7만'
            elif v <= 100000: return '7만~10만'
            elif v <= 200000: return '10만~20만'
            else: return '20만 초과'
        except:
            return None
    df['월정료 구간'] = df['KTT월정료'].apply(fee_group)

    # ✅ 19. 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 20. 특수문자 제거
    df = clean_text_columns(df)

    # ✅ 21. 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 22. 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ 23. CSV 저장
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"엑셀 저장 경로: {output_excel_path}")
    print(f"CSV 저장 경로: {output_csv_path}")

except Exception as e:
    print(f"\n❌ 오류 발생: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-10-3024f2342942>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype("object").apply(convert_date)
<ipython-input-10-3024f2342942>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype("object").apply(convert_date)
<ipython-input-10-3024f2342942>:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.


❌ 오류 발생: expected string or bytes-like object, got 'Series'


In [ ]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 필수 라이브러리
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re

# ✅ 3. 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 4. 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df.loc[:, col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?:;()\[\]{}@&%~\-+=/\\_|\'"<>※•·ㆍ→←↑↓▶☆★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 5. 날짜 변환 함수 (99991231 포함)
def convert_date(value):
    try:
        if pd.isna(value):
            return None
        value = str(value).strip()
        if value in ['', 'nan', '99991231']:
            return None
        if len(value) == 8 and value.isdigit():
            return f"{value[:4]}-{value[4:6]}-{value[6:8]}"
        return value if '-' in value else None
    except:
        return None

# ✅ 6. 경로 설정
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 7. 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 8. 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]].copy()
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 9. 본부명 통합
    df.loc[:, '관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 10. 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    for col in additional_columns:
        df.loc[:, col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 11. 제외사유 제거
    df = df[df['제외사유'].astype(str).str.strip() == '']

    # ✅ 12. 계약번호 중복 제거 및 월정료 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 13. 날짜 정제 (FutureWarning 완전 제거)
    date_cols = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
    for col in date_cols:
        if col in df.columns:
            df.loc[:, col] = df[col].astype("object").apply(convert_date)

    # ✅ 14. 정지희망종료일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime('%Y-%m-%d')
    df.loc[:, '정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x else x
    )

    # ✅ 15. 종료희망일 초과 여부
    df['종료희망일 초과 여부'] = df['정지희망종료일'].apply(
        lambda x: '초과' if pd.to_datetime(x, errors='coerce') <= last_day else '미초과'
    )

    # ✅ 16. 정지일수 계산
    def calc_days(start, end):
        try:
            s = pd.to_datetime(start, errors='coerce')
            e = pd.to_datetime(end, errors='coerce')
            return max((e - s).days + 1, 0)
        except:
            return None
    df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 17. 정지일수 구간
    df['정지일수 구간'] = pd.cut(df['정지일수'], bins=[-1, 89, 119, 149, float('inf')],
                                labels=['89일이하', '90~119일', '120~149일', '150일이상'])

    # ✅ 18. 월정료 구간
    def fee_group(val):
        try:
            v = float(str(val).replace(',', ''))
            if v <= 50000: return '5만 이하'
            elif v <= 70000: return '5만~7만'
            elif v <= 100000: return '7만~10만'
            elif v <= 200000: return '10만~20만'
            else: return '20만 초과'
        except:
            return None
    df['월정료 구간'] = df['KTT월정료'].apply(fee_group)

    # ✅ 19. 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 20. 특수문자 제거
    df = clean_text_columns(df)

    # ✅ 21. 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 22. 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ 23. CSV 저장
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"엑셀 저장 경로: {output_excel_path}")
    print(f"CSV 저장 경로: {output_csv_path}")

except Exception as e:
    print(f"\n❌ 오류 발생: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-11-9a89429cd27e>:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2025-04-01' '2024-11-30' '2024-10-24' ... '2025-03-31' '2025-03-31'
 '2025-04-24']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].astype("object").apply(convert_date)
<ipython-input-11-9a89429cd27e>:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2026-03-31' '2025-11-29' '2025-10-23' ... '2030-03-30' '2029-03-30'
 '2029-04-23']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].astype("object").apply(convert_date)
<ipython-input-11-9a89429cd27e>:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2024-10-31' '2025-04-01' '2025-04-01' ... '2025-04-02' '2025-0


❌ 오류 발생: expected string or bytes-like object, got 'Series'


In [ ]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 필수 라이브러리
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re

# ✅ 3. 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 4. 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df.loc[:, col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?:;()\[\]{}@&%~\-+=/\\_|\'"<>※•·ㆍ→←↑↓▶☆★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 5. 날짜 변환 함수
def convert_date(value):
    try:
        if pd.isna(value):
            return None
        value = str(value).strip()
        if value in ['', 'nan', '99991231']:
            return None
        if len(value) == 8 and value.isdigit():
            return f"{value[:4]}-{value[4:6]}-{value[6:8]}"
        return value if '-' in value else None
    except Exception:
        return None

# ✅ 6. 경로 설정
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 7. 파일 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 8. 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]].copy()
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 9. 본부명 정제
    df.loc[:, '관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 10. 추가 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    for col in additional_columns:
        df.loc[:, col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 11. 제외사유 제거
    df = df[df['제외사유'].astype(str).str.strip() == '']

    # ✅ 12. 계약번호 기준 월정료 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 13. 날짜 정제 (Series 오류 방지)
    date_cols = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
    for col in date_cols:
        if col in df.columns:
            df.loc[:, col] = df[col].astype("object").apply(convert_date)

    # ✅ 14. 정지희망종료일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime('%Y-%m-%d')
    df['정지희망종료일'] = df['정지희망종료일'].apply(lambda x: last_day_str if not x else x)

    # ✅ 15. 종료희망일 초과 여부
    df['종료희망일 초과 여부'] = df['정지희망종료일'].apply(
        lambda x: '초과' if pd.to_datetime(x, errors='coerce') <= last_day else '미초과'
    )

    # ✅ 16. 정지일수 계산
    def calc_days(start, end):
        try:
            s = pd.to_datetime(start, errors='coerce')
            e = pd.to_datetime(end, errors='coerce')
            return max((e - s).days + 1, 0)
        except:
            return None
    df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 17. 정지일수 구간
    df['정지일수 구간'] = pd.cut(df['정지일수'], bins=[-1, 89, 119, 149, float('inf')],
                                  labels=['89일이하', '90~119일', '120~149일', '150일이상'])

    # ✅ 18. 월정료 구간
    def fee_group(val):
        try:
            v = float(str(val).replace(',', ''))
            if v <= 50000: return '5만 이하'
            elif v <= 70000: return '5만~7만'
            elif v <= 100000: return '7만~10만'
            elif v <= 200000: return '10만~20만'
            else: return '20만 초과'
        except:
            return None
    df['월정료 구간'] = df['KTT월정료'].apply(fee_group)

    # ✅ 19. 컬럼 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 20. 특수문자 제거
    df = clean_text_columns(df)

    # ✅ 21. 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 22. 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ 23. CSV 저장
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"엑셀 경로: {output_excel_path}")
    print(f"CSV 경로: {output_csv_path}")

except Exception as e:
    print(f"\n❌ 오류 발생: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-12-ab17846172d2>:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2025-04-01' '2024-11-30' '2024-10-24' ... '2025-03-31' '2025-03-31'
 '2025-04-24']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].astype("object").apply(convert_date)
<ipython-input-12-ab17846172d2>:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2026-03-31' '2025-11-29' '2025-10-23' ... '2030-03-30' '2029-03-30'
 '2029-04-23']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = df[col].astype("object").apply(convert_date)
<ipython-input-12-ab17846172d2>:95: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2024-10-31' '2025-04-01' '2025-04-01' ... '2025-04-02' '2025-0


❌ 오류 발생: expected string or bytes-like object, got 'Series'


In [ ]:
# ✅ 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 필수 라이브러리
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re

# ✅ 3. 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 4. 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df.loc[:, col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?:;()\[\]{}@&%~\-+=/\\_|\'"<>※•·ㆍ→←↑↓▶☆★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 5. 안전한 날짜 변환 함수 (모든 오류 제거)
def convert_date_safe(value):
    if pd.isna(value) or value in ['', '99991231', 'nan', None]:
        return None
    try:
        value = str(int(float(value))).strip()
        if len(value) == 8 and value.isdigit():
            return f"{value[:4]}-{value[4:6]}-{value[6:8]}"
        elif '-' in value:
            return value
        else:
            return None
    except:
        return None

# ✅ 6. 경로 설정
file_path = '/content/drive/MyDrive/시설/정지/C155_03104_00045.csv'
additional_file_path = '/content/drive/MyDrive/시설/유지/3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = '/content/drive/MyDrive/시설/정지/2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 7. 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 8. 컬럼 필터 및 정제
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]].copy()
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 9. 본부명 통합
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 10. 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 11. 제외사유 제거
    df = df[df['제외사유'].astype(str).str.strip() == '']

    # ✅ 12. 계약번호 기준 중복 제거 및 월정료 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 13. 날짜 정제 (안전하게 적용)
    date_cols = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
    for col in date_cols:
        if col in df.columns:
            df[col] = df[col].astype("object").apply(convert_date_safe)

    # ✅ 14. 정지희망종료일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime('%Y-%m-%d')
    df['정지희망종료일'] = df['정지희망종료일'].apply(lambda x: last_day_str if not x else x)

    # ✅ 15. 종료희망일 초과 여부
    df['종료희망일 초과 여부'] = df['정지희망종료일'].apply(
        lambda x: '초과' if pd.to_datetime(x, errors='coerce') <= last_day else '미초과'
    )

    # ✅ 16. 정지일수 계산
    def calc_days(start, end):
        try:
            s = pd.to_datetime(start, errors='coerce')
            e = pd.to_datetime(end, errors='coerce')
            return max((e - s).days + 1, 0)
        except:
            return None
    df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 17. 정지일수 구간
    df['정지일수 구간'] = pd.cut(df['정지일수'], bins=[-1, 89, 119, 149, float('inf')],
                                labels=['89일이하', '90~119일', '120~149일', '150일이상'])

    # ✅ 18. 월정료 구간
    def fee_group(val):
        try:
            v = float(str(val).replace(',', ''))
            if v <= 50000: return '5만 이하'
            elif v <= 70000: return '5만~7만'
            elif v <= 100000: return '7만~10만'
            elif v <= 200000: return '10만~20만'
            else: return '20만 초과'
        except:
            return None
    df['월정료 구간'] = df['KTT월정료'].apply(fee_group)

    # ✅ 19. 컬럼 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 20. 특수문자 제거
    df = clean_text_columns(df)

    # ✅ 21. 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 22. 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ 23. CSV 저장
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"엑셀 경로: {output_excel_path}")
    print(f"CSV 경로: {output_csv_path}")

except Exception as e:
    print(f"\n❌ 오류 발생: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-13-8ef5054c2f2a>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype("object").apply(convert_date_safe)
<ipython-input-13-8ef5054c2f2a>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype("object").apply(convert_date_safe)



❌ 오류 발생: expected string or bytes-like object, got 'Series'


In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import re
import os

# ✅ 인코딩 자동 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        return 'cp949' if encoding and 'euc' in encoding.lower() else encoding or 'utf-8'

# ✅ 특수문자 제거 함수
def clean_text_columns(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).apply(
            lambda x: re.sub(r'[^\x00-\x7F가-힣a-zA-Z0-9\s,.!?()\[\]{}@&%~\-+=/\\_|\'"<>※•·ㆍ→←↑↓▶☆★○●◎◇◆□■♠♥♣♦¤°▪️◾◽️✔️❌✅]+', '', x)
        )
    return df

# ✅ 날짜 변환 함수 (안정형)
def convert_date_safe(value):
    if pd.isna(value) or value in ['', '99991231', 'nan', None]:
        return None
    try:
        value = str(int(float(value))).strip()
        if len(value) == 8 and value.isdigit():
            return f"{value[:4]}-{value[4:6]}-{value[6:8]}"
        elif '-' in value:
            return value
        else:
            return None
    except:
        return None

# ✅ 파일 경로 설정 (Windows용)
file_path = r'D:\시설\정지\C155_03104_00045.csv'
additional_file_path = r'D:\시설\유지\3월마감 조건추출 선택컬럼_2403 2.csv'
output_excel_path = r'D:\시설\정지\2025년0425_전사 정지조건리스트_완성.xlsx'
output_csv_path = r'D:\시설\정지\2025년0425_전사 정지조건리스트_완성.csv'

try:
    # ✅ 파일 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    add_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터
    main_cols = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in main_cols if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({'강원본부': '강북/강원본부', '서부본부': '강남/서부본부'})

    # ✅ 추가 컬럼 병합
    add_cols = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    add_df = add_df.drop_duplicates(subset=['계약번호'])
    for col in add_cols:
        df[col] = df['계약번호'].map(add_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    df = df[df['제외사유'].astype(str).str.strip() == '']

    # ✅ 월정료 합산 및 중복 제거
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 날짜 정제
    date_cols = ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']
    for col in date_cols:
        if col in df.columns:
            df[col] = df[col].astype("object").apply(convert_date_safe)

    # ✅ 정지희망종료일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime('%Y-%m-%d')
    df['정지희망종료일'] = df['정지희망종료일'].apply(lambda x: last_day_str if not x else x)

    # ✅ 종료희망일 초과 여부
    df['종료희망일 초과 여부'] = df['정지희망종료일'].apply(
        lambda x: '초과' if pd.to_datetime(x, errors='coerce') <= last_day else '미초과'
    )

    # ✅ 정지일수 계산
    def calc_days(start, end):
        try:
            s = pd.to_datetime(start, errors='coerce')
            e = pd.to_datetime(end, errors='coerce')
            return max((e - s).days + 1, 0)
        except:
            return None
    df['정지일수'] = df.apply(lambda row: calc_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    df['정지일수 구간'] = pd.cut(df['정지일수'], bins=[-1, 89, 119, 149, float('inf')],
                                  labels=['89일이하', '90~119일', '120~149일', '150일이상'])

    # ✅ 월정료 구간
    def fee_group(val):
        try:
            v = float(str(val).replace(',', ''))
            if v <= 50000: return '5만 이하'
            elif v <= 70000: return '5만~7만'
            elif v <= 100000: return '7만~10만'
            elif v <= 200000: return '10만~20만'
            else: return '20만 초과'
        except:
            return None
    df['월정료 구간'] = df['KTT월정료'].apply(fee_group)

    # ✅ 컬럼 순서 정리
    final_cols = add_cols + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + main_cols
    df = df[[col for col in final_cols if col in df.columns]]

    # ✅ 특수문자 제거
    df = clean_text_columns(df)

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 강조색 적용
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(add_cols) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow
    wb.save(output_excel_path)

    # ✅ CSV 저장
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"\n✅ 처리 완료! 행 수: {df.shape[0]}")
    print(f"저장 위치:\n- 엑셀: {output_excel_path}\n- CSV: {output_csv_path}")

except Exception as e:
    print(f"\n❌ 오류 발생: {e}")

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250422\C150_G0000_00013.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250422\2025년0422_전사 정지조건리스트_완성_서비스번호.csv'

try:
    # ✅ 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 통일: 'KTT월정료(조정)' → 'KTT월정료' 우선 적용
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''  # 없는 경우 빈 컬럼 생성

    # ✅ 날짜 포맷 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간 분류
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만원 이하'
            elif amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글 호환용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"✅ 최종 저장 완료! (행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)

In [ ]:
import win32com.client as win32
import pandas as pd
import os

# 1. 엑셀 열려 있는 인스턴스 연결
excel = win32.GetActiveObject("Excel.Application")
wb = excel.ActiveWorkbook

# 2. 시트명 고정: "해지data"
sheet_name = "3월"
sheet_names = [sheet.Name for sheet in wb.Sheets]

if sheet_name not in sheet_names:
    print(f"❌ 시트명 '{sheet_name}'이 엑셀에 존재하지 않습니다.")
    print("📌 시트 목록:", sheet_names)
    exit()

print(f"▶️ 선택된 시트: {sheet_name}")

# 3. 시트 복사 → 임시 워크북 → CSV 저장
tmp_path = os.path.expanduser("D:\\시설\\유지\\2025\\3월\\_temp_3월마감.csv")  # ✅ 임시 파일로 수정
final_csv_path = os.path.expanduser("D:\\시설\\유지\\2025\\3월\\3월마감 조건추출 선택컬럼_2403.csv")

# 이후 동일...

ws = wb.Sheets(sheet_name)
ws.Copy()
new_wb = excel.ActiveWorkbook
new_wb.SaveAs(tmp_path, FileFormat=6)
new_wb.Close(False)
print(f"✅ 임시 CSV 저장 완료: {tmp_path}")

# 4. 인코딩 자동 감지 후 읽기
encodings = ["utf-8-sig", "cp949", "euc-kr", "utf-8"]
df = None
for enc in encodings:
    try:
        df = pd.read_csv(tmp_path, encoding=enc)
        print(f"✅ 인코딩 성공: {enc}")
        break
    except UnicodeDecodeError:
        print(f"❌ 인코딩 실패: {enc}")

if df is None:
    print("❌ 인코딩 실패로 종료합니다.")
    exit()

# 5. 컬럼 미리보기 및 사용자 선택
print("\n📋 시트 컬럼 목록:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

col_input = input("\n✅ 저장할 컬럼 번호를 쉼표로 입력 (예: 1,3,5): ")
selected_indices = [int(i.strip()) - 1 for i in col_input.split(",") if i.strip().isdigit()]
selected_columns = [df.columns[i] for i in selected_indices if i < len(df.columns)]

if not selected_columns:
    print("⚠️ 선택한 컬럼이 없습니다. 종료합니다.")
    os.remove(tmp_path)
    exit()

# 6. 필터에 필요한 컬럼 체크
required_cols = ["관리본부명", "서비스(대)"]
for col in required_cols:
    if col not in df.columns:
        print(f"⚠️ 필터 조건에 필요한 컬럼 '{col}' 이 존재하지 않습니다.")
        os.remove(tmp_path)
        exit()

# 7. 조건 필터 적용
# df["KTT월정료(조정)"] = pd.to_numeric(df["월정료조정"], errors="coerce")

filtered_df = df[
#    (df["요금구분"] == "대상") &
#    (df["채널구분"] == "SP") &
#   (df["월정료조정"] >= 100000) &
#   (df["관리본부명"].isin(["강북/강원본부", "강원본부"])) &
    (df["서비스(대)"] == "기본서비스")
]

# 8. 선택한 컬럼만 저장
if not filtered_df.empty:
    filtered_df[selected_columns].to_csv(final_csv_path, index=False, encoding="utf-8-sig")
    print(f"\n✅ 조건 적용 + 선택 컬럼만 CSV 저장 완료:\n{final_csv_path}")
else:
    print("\n⚠️ 조건을 만족하는 데이터가 없습니다. 파일 저장 생략됨.")

# 9. 임시 파일 삭제
os.remove(tmp_path)

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250425\C155_03104_00045.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250425\2025년0425_전사 정지조건리스트_완성_중복제거0425.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250425\2025년0425_전사 정지조건리스트_완성_중복제거0425.csv'

try:
    # ✅ 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 통일: 'KTT월정료(조정)' → 'KTT월정료' 우선 적용
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''  # 없는 경우 빈 컬럼 생성

    # ✅ 날짜 포맷 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간 분류
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만원 이하'
            elif amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글 호환용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"✅ 최종 저장 완료! (행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import os

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 경로 설정
file_path = r'D:\시설\정지\일일정지\20250428\C150_G0000_00012.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250428\2025년0428_전사 정지조건리스트_완성_중복제거0428.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250428\2025년0428_전사 정지조건리스트_완성_중복제거0428.csv'

try:
    # ✅ CSV 로드
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 통일
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce').fillna(0)

    # ✅ 날짜 포맷 정제 함수
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간 분류
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만원 이하'
            elif amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 계약번호 기준 중복 제거 + KTT월정료 합산
    df = df.groupby('계약번호', as_index=False).agg({
        **{col: 'first' for col in df.columns if col not in ['계약번호', 'KTT월정료']},
        'KTT월정료': 'sum'
    })

    # ✅ 컬럼 정렬
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 강조 색상 적용 (노란색)
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"📄 엑셀 저장 경로: {output_excel_path}")
    print(f"📄 CSV 저장 경로: {output_csv_path}")

except Exception as e:
    print("❌ 오류 발생:", e)

❌ 오류 발생: [Errno 2] No such file or directory: 'D:\\시설\\정지\\일일정지\\20250428\\C150_G0000_00012.csv'


In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet
import os

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'

# ✅ 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250428\C150_G0000_00012.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250428\2025년0428_전사 정지조건리스트_완성_중복제거0428.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250428\2025년0428_전사 정지조건리스트_완성_중복제거0428.csv'

try:
    # ✅ 데이터 로드
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 필수 컬럼 정리
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 통일
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 정보 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ KTT월정료 통일 및 숫자화
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    df['KTT월정료'] = pd.to_numeric(df['KTT월정료'], errors='coerce').fillna(0)

    # ✅ 날짜 포맷 정제
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 계약번호 기준 중복 제거 + KTT월정료 합산
    df = df.groupby('계약번호', as_index=False).agg({
        **{col: 'first' for col in df.columns if col not in ['계약번호', 'KTT월정료']},
        'KTT월정료': 'sum'
    })

    # ✅ 월정료 구간 재계산
    def categorize_fee(val):
        try:
            if pd.isna(val): return None
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만원 이하'
            elif amount <= 70000:
                return '5만원 초과 ~ 7만원 이하'
            elif amount <= 100000:
                return '7만원 초과 ~ 10만원 이하'
            elif amount <= 200000:
                return '10만원 초과 ~ 20만원 이하'
            else:
                return '20만원 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 강조 색상 (노란색)
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (한글용)
    df.to_csv(output_csv_path, index=False, encoding='cp949')

    print(f"\n✅ 최종 저장 완료! 행 수: {df.shape[0]}")
    print(f"📄 엑셀 저장 경로: {output_excel_path}")
    print(f"📄 CSV 저장 경로: {output_csv_path}")

except Exception as e:
    print("❌ 오류 발생:", e)

❌ 오류 발생: [Errno 2] No such file or directory: 'D:\\시설\\정지\\일일정지\\20250428\\C150_G0000_00012.csv'


In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from datetime import datetime
import calendar
import chardet

# ✅ 인코딩 감지 함수
def detect_encoding(file_path, max_bytes=100000):
    with open(file_path, 'rb') as f:
        sample = f.read(max_bytes)
        result = chardet.detect(sample)
        encoding = result['encoding']
        if encoding and 'euc' in encoding.lower():
            encoding = 'cp949'
        return encoding or 'cp949'


# ✅ 파일 경로 설정
file_path = r'D:\시설\정지\일일정지\20250422\C150_G0000_00012.csv'
additional_file_path = r"D:\시설\유지\2025\3월\3월마감 조건추출 선택컬럼_2403.csv"
output_excel_path = r'D:\시설\정지\일일정지\20250428\2025년0428_전사 정지조건리스트_완성_중복제거0428.xlsx'
output_csv_path = r'D:\시설\정지\일일정지\20250428\2025년0428_전사 정지조건리스트_완성_중복제거0428.csv'


try:
    # ✅ 데이터 불러오기
    df = pd.read_csv(file_path, encoding=detect_encoding(file_path), low_memory=False)
    additional_df = pd.read_csv(additional_file_path, encoding=detect_encoding(additional_file_path), low_memory=False)

    # ✅ 주요 컬럼 필터링
    columns_to_extract = [
        '관리본부명', '관리지사명', '고객번호', '계약번호', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)',
        '상호', '고객구분', '사업용구분', '계약상태(대)', '설치주소', 'KTT월정료(조정)', 'KTT월정료',
        '계약시작일', '계약종료일', '영업자명', '정지시작일자', '정지희망종료일', '계약최초서비스게시일', '영업구역정보'
    ]
    df = df[[col for col in columns_to_extract if col in df.columns]]
    df = df[(df['서비스(대)'] == '기본서비스') & (df['사업용구분'] != '사업용')]

    # ✅ 본부명 정제
    df['관리본부명'] = df['관리본부명'].replace({
        '강원본부': '강북/강원본부',
        '서부본부': '강남/서부본부'
    })

    # ✅ 추가 컬럼 병합
    additional_columns = ['시설구분', '요금구분', '제외사유', '매출구분', '실적채널', '고알프', '설치주소']
    additional_df = additional_df.drop_duplicates(subset=['계약번호'])
    additional_df = additional_df[['계약번호'] + [col for col in additional_columns if col in additional_df.columns]]

    for col in additional_columns:
        df[col] = df['계약번호'].map(additional_df.set_index('계약번호')[col]).fillna('')

    # ✅ 제외사유 제거
    if '제외사유' in df.columns:
        df['제외사유'] = df['제외사유'].astype(str).str.strip()
        df = df[df['제외사유'] == '']

    # ✅ 계약번호 기준 중복 제거 및 KTT월정료(조정) 합산
    if 'KTT월정료(조정)' in df.columns:
        df['KTT월정료(조정)'] = pd.to_numeric(df['KTT월정료(조정)'].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
        agg_dict = {col: 'first' for col in df.columns if col != 'KTT월정료(조정)'}
        agg_dict['KTT월정료(조정)'] = 'sum'
        df = df.groupby('계약번호', as_index=False).agg(agg_dict)
        df['KTT월정료'] = df['KTT월정료(조정)']
    elif 'KTT월정료' not in df.columns:
        df['KTT월정료'] = ''

    # ✅ 날짜 정제
    def convert_date(date_str):
        try:
            if pd.isna(date_str) or not str(date_str).strip():
                return None
            date_str = str(date_str).strip()
            if len(date_str) >= 8:
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
            return None
        except:
            return None

    for col in ['계약시작일', '계약종료일', '정지시작일자', '정지희망종료일', '계약최초서비스게시일']:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(convert_date)

    # ✅ 종료희망일 보정
    today = datetime.today()
    last_day = datetime(today.year, today.month, calendar.monthrange(today.year, today.month)[1])
    last_day_str = last_day.strftime("%Y-%m-%d")
    df['정지희망종료일'] = df['정지희망종료일'].apply(
        lambda x: last_day_str if not x or x == '9999-12-31' else x
    )

    # ✅ 종료희망일 초과 여부
    def check_exceeding(row):
        try:
            end_date = datetime.strptime(row['정지희망종료일'], '%Y-%m-%d')
            return '초과' if end_date <= last_day else '미초과'
        except:
            return None

    df['종료희망일 초과 여부'] = df.apply(check_exceeding, axis=1)

    # ✅ 정지일수 계산
    def calculate_freeze_days(start, end):
        try:
            if not start or not end:
                return None
            start = datetime.strptime(start, "%Y-%m-%d")
            end = datetime.strptime(end, "%Y-%m-%d")
            return max((end - start).days + 1, 0)
        except:
            return None

    df['정지일수'] = df.apply(lambda row: calculate_freeze_days(row['정지시작일자'], row['정지희망종료일']), axis=1)

    # ✅ 정지일수 구간
    def categorize_freeze_days(days):
        if pd.isna(days): return None
        elif days <= 89: return '89일이하'
        elif days <= 119: return '90~119일'
        elif days <= 149: return '120~149일'
        else: return '150일이상'

    df['정지일수 구간'] = df['정지일수'].apply(categorize_freeze_days)

    # ✅ 월정료 구간
    def categorize_fee(val):
        try:
            amount = float(str(val).replace(',', ''))
            if amount <= 50000:
                return '5만 이하'
            elif amount <= 70000:
                return '5만 ~ 7만 이하'
            elif amount <= 100000:
                return '7만 ~ 10만 이하'
            elif amount <= 200000:
                return '10만 ~ 20만 이하'
            elif amount <= 500000:
                return '20만 ~ 50만 이하'
            else:
                return '50만 초과'
        except:
            return None

    df['월정료 구간'] = df['KTT월정료'].apply(categorize_fee)

    # ✅ 컬럼 순서 정리
    final_columns = additional_columns + ['정지일수', '정지일수 구간', '종료희망일 초과 여부', '월정료 구간'] + columns_to_extract
    df = df[[col for col in final_columns if col in df.columns]]

    # ✅ 엑셀 저장
    df.to_excel(output_excel_path, index=False, engine='openpyxl')

    # ✅ 노란색 강조 처리
    wb = load_workbook(output_excel_path)
    ws = wb.active
    yellow_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
    for col in range(1, len(additional_columns) + 5):
        for row in range(2, ws.max_row + 1):
            ws.cell(row=row, column=col).fill = yellow_fill
    wb.save(output_excel_path)

    # ✅ CSV 저장 (UTF-8 with BOM — Excel 한글 호환)
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

    print(f"✅ 최종 저장 완료! (행 수: {df.shape[0]})")

except Exception as e:
    print("❌ 오류 발생:", e)

❌ 오류 발생: [Errno 2] No such file or directory: 'D:\\시설\\정지\\일일정지\\20250422\\C150_G0000_00012.csv'


In [ ]:
Sub ApplyStopDurationGroup()
    Dim ws As Worksheet
    Dim lastRow As Long
    Dim i As Long
    Dim value As Long

    Set ws = ActiveSheet
    lastRow = ws.Cells(ws.Rows.Count, "V").End(xlUp).Row ' V열의 마지막 행 찾기

    For i = 2 To lastRow ' 2행부터 시작 (헤더 제외)
        If IsNumeric(ws.Cells(i, "V").Value) Then
            value = ws.Cells(i, "V").Value

            Select Case value
                Case Is <= 90
                    ws.Cells(i, "AV").Value = "90일 이하"
                Case 91 To 119
                    ws.Cells(i, "AV").Value = "90일 이상"
                Case 120 To 149
                    ws.Cells(i, "AV").Value = "120일 이상"
                Case Is >= 150
                    ws.Cells(i, "AV").Value = "150일 이상"
            End Select
        Else
            ws.Cells(i, "AV").Value = "오류 또는 빈값"
        End If
    Next i

    MsgBox "정지일수 구간 분류 완료!"
End Sub